<a href="https://colab.research.google.com/github/Tahir7044/machine-learning/blob/master/tensorflow2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tensorflow-probability==0.7.0

In [0]:
pip install google-auth~=1.7.2

In [0]:
pip install requests~=2.21.0

In [0]:
pip install six~=1.12.0

In [0]:
pip install folium==0.2.1

In [0]:
pip install tensorflow

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
print(tf.version)

<module 'tensorflow_core._api.v2.version' from '/tensorflow-2.1.0/python3.6/tensorflow_core/_api/v2/version/__init__.py'>


In [0]:
st=tf.Variable("tahir",tf.string)
num=tf.Variable(123,tf.int16)
fl=tf.Variable(123.15,tf.float16)

In [0]:
print(tf.rank(st))

tf.Tensor(0, shape=(), dtype=int32)


In [0]:
rank1=tf.Variable([[12]],tf.int16)
rank2=tf.Variable([[12,45,65,4],
                   [1,2,4,7],
                   [5,6,7,8]],tf.int16)
print(tf.rank(rank1))
print(tf.rank(rank2))
print(rank1.shape)
print(rank2.shape)

In [0]:
tensor=tf.ones([2,2,3],dtype='int16')
tensor

In [0]:
tensor1=tf.reshape(tensor,[3,2,2])
tensor2=tf.reshape(tensor,[4,-1])
tensor3=tf.reshape(tensor,[2,3,-1])
print(tensor1)
print(tensor2)
print(tensor3)

path setting and dataset downloading



In [0]:
pip install -q sklearn

In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc

In [0]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!kaggle competitions download -c titanic -p /content

**algorithm**
*   linear regression

In [0]:
data_set=pd.read_csv("titanic_train.csv")

In [0]:
data_set.shape

(891, 12)

In [0]:
col=data_set.columns
print(col)

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [0]:
data_test=pd.read_csv("titanic_test.csv")
train_y=data_set.pop('Survived')

In [0]:
train_y.shape

(891,)

In [0]:
data_set['Cabin'].value_counts()

In [0]:
plt.hist(x='Pclass',bins=5,data=data_set);
plt.show()

In [0]:
catagorical_col=['SibSp','Pclass','Sex', 'Parch','Cabin','Embarked']
numerical_col=['Age','Fare']

In [0]:
feature_col=[]
for ft_name in catagorical_col:
  uniq_data=data_set[ft_name].unique()
  feature_col.append(tf.feature_column.categorical_column_with_vocabulary_list(ft_name,uniq_data))
for ft_name in numerical_col:
  feature_col.append(tf.feature_column.numeric_column(ft_name,dtype=tf.float32))

In [0]:
print(data_set['Embarked'].unique())

['S' 'C' 'Q' nan]


In [0]:
print(tf.feature_column.categorical_column_with_vocabulary_list('tahir',['yes','no']))
print(tf.feature_column.numeric_column('algo',dtype=tf.float32))

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_set,train_y , test_size=0.25, random_state=42)

In [0]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(668, 11) (668,)
(223, 11) (223,)


In [0]:
X_train.isna().sum()

In [0]:
X_train['Age']=X_train['Age'].interpolate(method ='linear', limit_direction ='forward') 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
def make_input_fun(data_df,label_df,n_epoc=10,shuffle=True,batch_size=32):
  def input_function():
    ds=tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds=ds.shuffle(1000)
    ds=ds.batch(batch_size).repeat(n_epoc)
    return ds
  return input_function
train_data=make_input_fun(X_train,y_train)
test_data=make_input_fun(X_test,y_test,n_epoc=1,shuffle=False)

In [0]:
linear_est=tf.estimator.LinearClassifier(feature_columns=feature_col)
linear_est.train(train_data)
result=linear_est.evaluate(test_data)
clear_output()
print(result['accuracy'])